In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import LabelEncoder, StandardScaler
label = LabelEncoder()
Ss = StandardScaler()
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


import warnings
warnings.filterwarnings('ignore')

In [38]:
train = pd.read_csv('train_s3TEQDk.csv')
test = pd.read_csv('test_mSzZ8RL.csv')

In [39]:
train.shape, test.shape

((245725, 11), (105312, 10))

In [40]:
test.isna().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         12522
Avg_Account_Balance        0
Is_Active                  0
dtype: int64

In [41]:
train['Credit_Product'].value_counts()

No     144357
Yes     72043
Name: Credit_Product, dtype: int64

In [42]:
train['Credit_Product'] = train['Credit_Product'].fillna(train['Credit_Product'].mode()[0])
test['Credit_Product'] = test['Credit_Product'].fillna(test['Credit_Product'].mode()[0])

In [43]:
cols = ['Age','Occupation', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active']

for i in cols:
    train['Gender'+str(i)] = train['Gender'].astype(str)+'_'+train[i].astype(str)
    test['Gender'+str(i)] = test['Gender'].astype(str)+'_'+test[i].astype(str)

In [44]:
cols = ['Occupation','Avg_Account_Balance','Is_Active']

for i in cols:
    train['Age'+str(i)] = train['Age'].astype(str)+'_'+train[i].astype(str)
    test['Age'+str(i)] = test['Age'].astype(str)+'_'+test[i].astype(str)

In [45]:
cols = ['Occupation', 'Channel_Code', 'Vintage','Credit_Product']

for i in cols:
    train['Region_Code'+str(i)] = train['Region_Code'].astype(str)+'_'+train[i].astype(str)
    test['Region_Code'+str(i)] = test['Region_Code'].astype(str)+'_'+test[i].astype(str)

In [46]:
cols = ['Channel_Code','Vintage','Credit_Product','Avg_Account_Balance','Is_Active']

for i in cols:
    train['Occupation'+str(i)] = train['Occupation'].astype(str)+'_'+train[i].astype(str)
    test['Occupation'+str(i)] = test['Occupation'].astype(str)+'_'+test[i].astype(str)

In [47]:
cols = ['Vintage','Credit_Product','Avg_Account_Balance','Is_Active']

for i in cols:
    train['Channel_Code'+str(i)] = train['Channel_Code'].astype(str)+'_'+train[i].astype(str)
    test['Channel_Code'+str(i)] = test['Channel_Code'].astype(str)+'_'+test[i].astype(str)

In [48]:
cols = ['Credit_Product','Avg_Account_Balance','Is_Active']

for i in cols:
    train['Vintage'+str(i)] = train['Vintage'].astype(str)+'_'+train[i].astype(str)
    test['Vintage'+str(i)] = test['Vintage'].astype(str)+'_'+test[i].astype(str)

In [49]:
cols = ['Avg_Account_Balance','Is_Active']

for i in cols:
    train['Credit_Product'+str(i)] = train['Credit_Product'].astype(str)+'_'+train[i].astype(str)
    test['Credit_Product'+str(i)] = test['Credit_Product'].astype(str)+'_'+test[i].astype(str)

In [50]:
cols = ['Is_Active']

for i in cols:
    train['Avg_Account_Balance'+str(i)] = train['Avg_Account_Balance'].astype(str)+'_'+train[i].astype(str)
    test['Avg_Account_Balance'+str(i)] = test['Avg_Account_Balance'].astype(str)+'_'+test[i].astype(str)

In [51]:
train = pd.get_dummies(train, columns = ['Gender','Region_Code','Occupation','Channel_Code'])

In [52]:
test = pd.get_dummies(test, columns = ['Gender','Region_Code','Occupation','Channel_Code'])
test.shape

(105312, 78)

In [53]:
train1 = pd.read_csv('train_s3TEQDk.csv')
test1 = pd.read_csv('test_mSzZ8RL.csv')

train['Gender'] = train1['Gender']
train['Region_Code'] = train1['Region_Code']
train['Occupation'] = train1['Occupation']
train['Channel_Code'] = train1['Channel_Code']


test['Gender'] = test1['Gender']
test['Region_Code'] = test1['Region_Code']
test['Occupation'] = test1['Occupation']
test['Channel_Code'] = test1['Channel_Code']

In [54]:
for col in train.columns:
    if (train[col].dtypes == 'object'):
        #Train[col] = Train[col].astype(str)
        train[col] = label.fit_transform(train[col])

In [55]:
for col in test.columns:
    if (test[col].dtypes == 'object'):
        #Test[col] = Test[col].astype(str)
        test[col] = label.fit_transform(test[col])

In [56]:
for cols in ['ID','Gender','Age','Region_Code','Occupation',
             'Channel_Code','Vintage','Credit_Product',
             'Avg_Account_Balance','Is_Active']:
    val_counts = train[cols].value_counts().to_dict()
    train[str(cols)+'counts'] = train[cols].map(val_counts) 

In [57]:
for cols in ['ID','Gender','Age','Region_Code','Occupation',
             'Channel_Code','Vintage','Credit_Product',
             'Avg_Account_Balance','Is_Active']:
    val_counts = test[cols].value_counts().to_dict()
    test[str(cols)+'counts'] = test[cols].map(val_counts) 

In [58]:
train.columns

Index(['ID', 'Age', 'Vintage', 'Credit_Product', 'Avg_Account_Balance',
       'Is_Active', 'Is_Lead', 'GenderAge', 'GenderOccupation',
       'GenderCredit_Product', 'GenderAvg_Account_Balance', 'GenderIs_Active',
       'AgeOccupation', 'AgeAvg_Account_Balance', 'AgeIs_Active',
       'Region_CodeOccupation', 'Region_CodeChannel_Code',
       'Region_CodeVintage', 'Region_CodeCredit_Product',
       'OccupationChannel_Code', 'OccupationVintage',
       'OccupationCredit_Product', 'OccupationAvg_Account_Balance',
       'OccupationIs_Active', 'Channel_CodeVintage',
       'Channel_CodeCredit_Product', 'Channel_CodeAvg_Account_Balance',
       'Channel_CodeIs_Active', 'VintageCredit_Product',
       'VintageAvg_Account_Balance', 'VintageIs_Active',
       'Credit_ProductAvg_Account_Balance', 'Credit_ProductIs_Active',
       'Avg_Account_BalanceIs_Active', 'Gender_Female', 'Gender_Male',
       'Region_Code_RG250', 'Region_Code_RG251', 'Region_Code_RG252',
       'Region_Code_RG253', '

In [59]:
for i in ['GenderAge', 'GenderOccupation','GenderIs_Active', 
          'AgeOccupation','AgeAvg_Account_Balance', 'AgeIs_Active',
          'Region_CodeOccupation','Region_CodeChannel_Code',
          'Region_CodeVintage','Region_CodeCredit_Product', 
          'OccupationChannel_Code','OccupationVintage', 
          'OccupationCredit_Product','OccupationAvg_Account_Balance', 
          'OccupationIs_Active','Channel_CodeVintage', 'Channel_CodeCredit_Product',
          'Channel_CodeAvg_Account_Balance', 'Channel_CodeIs_Active',
          'VintageCredit_Product', 'VintageAvg_Account_Balance',
          'VintageIs_Active', 'Credit_ProductAvg_Account_Balance',
          'Credit_ProductIs_Active', 'Avg_Account_BalanceIs_Active']:
    
    for j in ['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
              'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active']:
        
        train['mean'+str(j)+'per'+str(i)] = train.groupby([j])[i].transform('mean')
        test['mean'+str(j)+'per'+str(i)] = test.groupby([j])[i].transform('mean')
        
        train['std'+str(j)+'per'+str(i)] = train.groupby([j])[i].transform('sum')
        test['std'+str(j)+'per'+str(i)] = test.groupby([j])[i].transform('sum')

In [60]:
print('before removing duplicates from train',train.shape)
print('before removing duplicates from test',test.shape)
train = train[~train.index.duplicated(keep = 'first')].reset_index(drop=True)
test = test[~test.index.duplicated(keep = 'first')].reset_index(drop=True)
print('after removing duplicates from train',train.shape)
print('after removing duplicates from test',test.shape)

before removing duplicates from train (245725, 543)
before removing duplicates from test (105312, 542)
after removing duplicates from train (245725, 543)
after removing duplicates from test (105312, 542)


In [61]:
X = train.drop(['Is_Lead'], axis = 1)
y = train[['Is_Lead']]
y.head()

,Is_Lead
0,0
1,0
2,0
3,0
4,0


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 19278, stratify = y)

In [63]:
X_train = Ss.fit_transform(X_train)
X_test  = Ss.fit_transform(X_test)

In [64]:
rmse_list = []
accuracy_ist = []
f1_scores = []

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier





log = LogisticRegression()
KNN = KNeighborsClassifier()
NB = GaussianNB() 
SVM = SVC()
DT = DecisionTreeClassifier(criterion='gini', splitter='random', 
                           max_depth=5000, min_samples_split=25, min_samples_leaf=15, 
                           min_weight_fraction_leaf=0.0, max_features=None, 
                           random_state=645, max_leaf_nodes=None, 
                           min_impurity_decrease=0, min_impurity_split=None, 
                           ccp_alpha=0)
RF = RandomForestClassifier(n_estimators=5000, criterion='gini', max_depth=None, 
                             min_samples_split=25, min_samples_leaf=15, min_weight_fraction_leaf=0.0, 
                             max_features='log2', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                             min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=-1, 
                             random_state=645, verbose=0, warm_start=False, 
                             ccp_alpha=0.0, max_samples=None)

BC = BaggingClassifier(base_estimator = DT, n_estimators = 80)
GBR = GradientBoostingClassifier()
xgb = XGBClassifier()
SGD = SGDClassifier()
ADC = AdaBoostClassifier()

In [32]:
log.fit(X_train, y_train)
y_pred = log.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse2 = np.sqrt(MSE)
print('rmse is:    ',rmse2)
f1_score2 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score2)

MSE:      0.2151388747583681
rmse is:     0.46383065310344473
f1_score is:      0.31729837928585264


In [33]:
KNN.fit(X_train, y_train)
y_pred = KNN.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse3 = np.sqrt(MSE)
print('rmse is:    ',rmse3)
f1_score3 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score3)

MSE:      0.2357106521517957
rmse is:     0.4855004141623318
f1_score is:      0.3992947521261149


In [34]:
DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse6 = np.sqrt(MSE)
print('rmse is:    ',rmse6)
f1_score6 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score6)

MSE:      0.23371655305727948
rmse is:     0.4834423989031987
f1_score is:      0.4130812468063362


In [35]:
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse7 = np.sqrt(MSE)
print('rmse is:    ',rmse7)
f1_score7 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score7)

MSE:      0.20834265947705768
rmse is:     0.4564456807518915
f1_score is:      0.3826349110642147


In [ ]:
GBR.fit(X_train, y_train)
y_pred = GBR.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse9 = np.sqrt(MSE)
print('rmse is:    ',rmse9)
f1_score9 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score9)

In [ ]:
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse10 = np.sqrt(MSE)
print('rmse is:    ',rmse10)
f1_score10 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score10)

In [ ]:
SGD.fit(X_train, y_train)
y_pred = SGD.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse11 = np.sqrt(MSE)
print('rmse is:    ',rmse11)
f1_score11 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score11)

In [ ]:
ADC.fit(X_train, y_train)
y_pred = ADC.predict(X_test)
MSE = metrics.mean_squared_error(y_test,y_pred)
print('MSE:     ', MSE)
rmse12 = np.sqrt(MSE)
print('rmse is:    ',rmse12)
f1_score12 = f1_score(y_test, y_pred)
print("f1_score is:     ", f1_score12)

In [ ]:
f1_scores = [f1_score2,f1_score3,f1_score4,f1_score5,f1_score6,f1_score7,f1_score8,
             f1_score9,f1_score10,f1_score11,f1_score12]

rmse_list = [rmse2,rmse3,rmse4,rmse5,rmse6,rmse7,rmse8,rmse9,rmse10,rmse11,rmse12]

In [ ]:
evaluation = pd.DataFrame({'Model': Algo_names,'f1_score' : f1_scores, 'RMSE': rmse_list})
evaluation.sort_values(by = 'f1_score', ascending = False)

In [ ]:
submission = pd.read_csv('sample.csv')
model = RF
model.fit(X_train, y_train)
pred = model.predict(test)
Pred1 = np.where(pred > 0.5, "Yes", "No")
submission['Prediction'] = Pred1
submission.to_csv('RF72%model.csv', index = False)